In [62]:
#%pip install selenium
# https://chromedriver.chromium.org/downloads

import time
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select

def preparar_driver(path_projeto):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option('prefs', 
        {
            "plugins.always_open_pdf_externally": True,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            'download.default_directory': path_projeto.replace('/', '\\') + '\\arquivos'
        }
    )
    driver = webdriver.Chrome(service=Service(f'{path_projeto}/chromedriver.exe'), options=chrome_options)
    driver.maximize_window()

    login_url = 'https://painelprc.trf5.jus.br/painelprc/auth'
    driver.get(login_url)

    time.sleep(5)
    driver.find_element(By.ID, 'btnAutenticar').click()

    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.find_element(By.ID, 'cardextratodemo').find_element(By.TAG_NAME, 'button').click()

    time.sleep(3)
    driver.find_element(By.ID, 'form:numeroRequisitorio').clear()
    return driver

def download_demonstrativo(driver, path_arquivos, requisitorio):

    print(f'Buscando requisitório nº {requisitorio}')

    nome_arquivo = f'{path_arquivos}/{requisitorio}.pdf'
    if os.path.isfile(nome_arquivo): return 'já existe'

    driver.find_element(By.ID, 'form:numeroRequisitorio').clear()
    driver.find_element(By.ID, 'form:numeroRequisitorio').send_keys(requisitorio)
    driver.find_element(By.CSS_SELECTOR, '#form\:idTipoRequisitorio > tbody > tr > td:nth-child(3) > div').click()

    time.sleep(1)
    driver.find_element(By.ID, 'form:j_idt65').click()
    time.sleep(3)

    tabela_resultados = driver.find_element(By.ID, 'outroForm:listaextratoXXT_data')
    qtd_celulas_tabela_resultado = len(tabela_resultados.find_elements(By.TAG_NAME, 'td'))
    if qtd_celulas_tabela_resultado == 1: return 'não encontrado'
    
    driver.find_element(By.ID, 'outroForm:listaextratoXXT:0:btnpdf').click()
    time.sleep(3)

    nome_padrao =  path_arquivos.replace("/", "\\") + '\\PRC'

    os.rename(nome_padrao, nome_arquivo)

    return 'baixado com sucesso'

In [1]:
import csv

arquivo_entrada = 'requisitorios.csv'
arquivo_saida = 'resultados.csv'

# Função para ler arquivo
def ler_arquivo(nome_arquivo):
    lista = []
    with open(arquivo_entrada, 'r', encoding='UTF-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            lista.append(row[0])

    return lista

# Função para salvar arquivo
def salvar(rows, arquivo_saida):
    with open(arquivo_saida, 'a',  encoding='UTF-8',  newline='') as f:
        writer = csv.writer(f, delimiter =';')
        for row in rows:
            writer.writerow(row)

# Abre e lê os arquivos
lista = ler_arquivo(arquivo_entrada)
resultados = ler_arquivo(arquivo_saida)

# Busca o último item feito
index_ultimo_feito = lista.index(resultados[-1])
lista = lista[index_ultimo_feito:]

In [60]:
path_projeto = 'C:/Projetos/Martorelli/Demonstrativos'
driver = preparar_driver(path_projeto)

In [ ]:
path_arquivos = f'{path_projeto}/arquivos'

resultados = []
errors = []
i = 0

for requisitorio in lista:
    time.sleep(0.5) # Delay entre iterações
    
    if i % 100 == 0: salvar(resultados[-100:], 'arquivo_saida.csv') # Salva o arquivo a cada 100 iterações
    
    try:
        resultados.append(f'{requisitorio} {download_demonstrativo(driver, path_arquivos, requisitorio)}')
    except Exception as e:
        erro = [requisitorio, type(e).__name__] # Salva item com erro e a descrição do erro
        salvar(erro, 'erros.csv')
        errors.append(erro)
    
    i+=1

In [ ]:
print('Pasta dos arquivos:')
for file in os.scandir(f'{path_projeto}/arquivos'):
    print(file.name)

In [5]:
lista = ler_arquivo(arquivo_entrada)
lista

['2022.80.00.002.202769',
 '2022.80.00.002.202770',
 '2022.80.00.002.202774',
 '2022.80.00.002.202773',
 '2022.80.00.002.202776',
 '2022.80.00.002.202771',
 '2022.80.00.002.202772',
 '2022.80.00.002.202777',
 '2022.80.00.002.200645',
 '2022.80.00.002.200646',
 '2022.80.00.002.200647',
 '2022.80.00.002.200648',
 '2022.80.00.002.200654',
 '2022.80.00.002.200651',
 '2022.80.00.002.200652',
 '2022.80.00.002.202552',
 '2022.80.00.002.202551',
 '2022.80.00.002.202548',
 '2022.80.00.002.202550',
 '2022.80.00.002.202549',
 '2022.80.00.002.202562',
 '2022.80.00.002.202557',
 '2022.80.00.002.202560',
 '2022.80.00.002.202556',
 '2022.80.00.002.202554',
 '2022.80.00.002.202558',
 '2022.80.00.002.202555',
 '2021.80.00.002.207687',
 '2021.80.00.002.207689',
 '2021.80.00.002.207685',
 '2021.80.00.002.207688',
 '2021.80.00.002.207693',
 '2021.80.00.002.207690',
 '2021.80.00.002.207691',
 '2021.80.00.002.207692',
 '2021.80.00.002.207696',
 '2021.80.00.002.207695',
 '2021.80.00.002.207680',
 '2021.80.00

In [17]:
def log_on_google(log_entity, log_text):

    import requests as re
    payload = {
        'entry.1662748811': log_entity,
        'entry.1806349460': log_text,
        'hud': True,
        'fvv': '1',
        'pageHistory': '0',
        'fbzx': '-5127016485562115995'
    }

    form_url = 'https://docs.google.com/forms/u/0/d/e/1FAIpQLSd6GXzvOPJ23nVShkFiIO5kO1ZSKKJKHqP4C0j-X5yPJ1gyng/formResponse'
    response = re.post(url=form_url, data=payload)
    return response.status_code

log_entity = 'Martorelli - Demonstrativo'
log_text = '200.201.202'
log_on_google(log_entity, log_text)


200

In [ ]:
#re.search('name="fbzx" value="(.*)">',page).group(1)